In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import sys
import os
import glob
import flopy
import math
import numbers
import matplotlib.image as mpimg
from math import pi

from scipy.interpolate import interp1d
import pandas as pd
import pickle
import shapefile
import matplotlib as mpl
from collections import OrderedDict
import subprocess as sup
from flopy.utils.triangle import Triangle as Triangle
import geopandas as gpd
from shapely.geometry import LineString,Point,Polygon,shape
import fiona
from shapely import speedups
speedups.disable()
from datetime import datetime

np.set_printoptions(precision=6)

logfunc = lambda e: np.log10(e)

def write_input_files(gwf,modelname):
    headfile = '{}.hds'.format(modelname)
    head_filerecord = [headfile]
    budgetfile = '{}.bud'.format(modelname)
    budget_filerecord = [budgetfile]
    saverecord, printrecord = [('HEAD', 'ALL'), ('BUDGET', 'ALL')], [('HEAD', 'ALL')]
    oc = flopy.mf6.modflow.mfgwfoc.ModflowGwfoc(gwf, pname='oc', saverecord=saverecord, head_filerecord=head_filerecord,
                                                budget_filerecord=budget_filerecord, printrecord=printrecord)

def get_data(modelname, workspace):
    fpth = os.path.join(workspace, modelname +'.hds')
    hds = flopy.utils.binaryfile.HeadFile(fpth)  
    times = hds.get_times()
    head = hds.get_data(totim=times[-1])
    
    fpth = os.path.join(workspace, modelname +'.bud')
    bud = flopy.utils.binaryfile.CellBudgetFile(fpth)
    #flowja = bud.get_data(text='FLOW-JA-FACE')[0][0][0]
    spd = bud.get_data(text='DATA-SPDIS')[0]
    chdflow = bud.get_data(text='CHD')[-1]
    #return(head)
    return(head, spd, chdflow)#, flowja)

def find_kji(cell,nlay,nrow,ncol): #cellid is zerobased
    import math
    cellid = cell - 1
    k = math.floor(cellid/(ncol*nrow)) # Zero based
    j = math.floor((cellid - k*ncol*nrow)/ncol) # Zero based
    i = cellid - k*ncol*nrow - j*ncol
    return(k,j,i) # ZERO BASED!

def find_cellid(k,j,i,nlay,nrow,ncol): # returns zero based cell id
    return(i + j*ncol + k*ncol*nrow)

def x_to_col(x, delr):
    return(int(x/delr))

def y_to_row(y, delc):
    return(int(y/delc))

def z_to_lay(z, delz, zmax):
    return(int((zmax - z)/delz))

def lay_to_z(botm, top, lay, icpl=0):
    pillar = botm[:,icpl]
    if lay != 0:
        cell_top = pillar[lay-1]
    if lay == 0:
        cell_top = top[icpl]
    cell_bot = pillar[lay]
    dz = cell_top - cell_bot
    z = cell_top - dz/2        
    return(z)

def ch_flow(chdflow):
    flow_in, flow_out = 0., 0.
    for j in range(len(chdflow)):        
        if chdflow[j][2]>0: flow_out += chdflow[j][2]
        if chdflow[j][2]<0: flow_in  += chdflow[j][2]      
    return((flow_in, flow_out))

def get_q_disu(spd, flowja, gwf, staggered):

    qx, qy, qz = flopy.utils.postprocessing.get_specific_discharge(spd, gwf)
    # if cross-connections, recalculate qx taking into account overlap areas
    if staggered:
        gp = d2d.get_gridprops_disu6()
        iac = gp["iac"]
        ja = gp["ja"]
        ihc = gp["ihc"]
        topbycell = gp["top"]
        botbycell = gp["bot"]
        hwva = gp["hwva"]
        iconn = -1
        icell = -1
        for il in iac:
            icell += 1
            qxnumer = 0.
            qxdenom = 0.
            for ilnbr in range(il):
                iconn += 1
                if ihc[iconn] == 2:
                    inbr = ja[iconn]
                    if (inbr == icell):
                        continue
                    dz = min(topbycell[icell], topbycell[inbr]) - max(botbycell[icell], botbycell[inbr])
                    qxincr = flowja[iconn] / (hwva[iconn] * dz)
                    # equal weight given to each face, but could weight by distance instead
                    if (inbr < icell):
                        qxnumer += qxincr
                    else:
                        qxnumer -= qxincr
                    qxdenom += 1.
            qx[icell] = qxnumer / qxdenom

    #print(len(spd))
    qmag, qdir = [], []
    for i in range(len(spd)):
        qmag.append(np.sqrt(qx[i]**2 + qy[i]**2 + qz[i]**2))
        qdir.append(math.degrees(math.atan(qz[i]/qx[i])))      
    return(qmag,qx,qy,qz,qdir)


def RCA(x,X):
    #x is a point that we want to see if it is in  the polygon
    #X is the vertices of the polygon
    dim=len(X)
    #print(dim)
    #we are going to run a line to the left and see is it intercepts the line between 2 vertices
    ct=0   
    for i in range(dim-1):
        #print(X[i])
        if X[i][1]<X[i+1][1]:
            if x[1]>X[i][1] and  x[1]<=X[i+1][1]:
                xx=X[i][0]+(x[1]-X[i][1])/(X[i+1][1]-X[i][1])*(X[i+1][0]-X[i][0])
                if xx >= x[0]:
                    ct= ct + 1
        else:
            if x[1]>X[i+1][1] and  x[1]<=X[i][1]:
                xx=X[i][0]+(x[1]-X[i][1])/(X[i+1][1]-X[i][1])*(X[i+1][0]-X[i][0])
                if xx >= x[0]:
                    ct= ct + 1           
    return(ct % 2)

def find_cell_disv(x, y, xcyc):
    dist_from_xy = [] # Finding cell id of most centred cell
    for i in range(len(xcyc)):
        dist_from_xy.append(np.sqrt((xcyc[i][0] - x)**2 + (xcyc[i][1] - y)**2))
    cell = np.argmin(dist_from_xy)
    cell_coords = (xcyc[cell][0], xcyc[cell][1])
    #print('Actual xy = %s %s, Cell is %i, Cell centre is %s' %(x,y,cell,cell_coords))
    return (cell, cell_coords)

def find_cell_dis(x, y, x0, y1, dx, dy):
    col = int((x - x0)/dx)
    row = int((y1 - y)/dy)
    cell_coords = (dx/2 + col*dx, (y1 - dy/2) - row * dy)
    #print('Actual xy = %s %s, Cell col is %i row is %i, Cell centre is %s' %(x,y,col,row,cell_coords))
    return (col, row, cell_coords)

def disucell_to_xyz(self, disu_cell): # zerobased
    disv_cell = self.cellid_disv.flatten()[self.cellid_disv.flatten()==disu_cell]    
    lay  = math.floor(disv_cell/(self.ncpl)) # Zero based
    icpl = math.floor(disv_cell - lay * self.ncpl) # Zero based
    x = self.vgrid.xcellcenters[icpl]
    y = self.vgrid.ycellcenters[icpl]
    z = self.vgrid.zcellcenters[lay, icpl]
    return(x,y,z)

def xyz_to_disucell(self, x, y, z): # zerobased
    self.vgrid = flopy.discretization.VertexGrid(ncpl = self.ncpl, vertices = self.vertices, cell2d = self.cell2d,
                                                 nlay = self.nlay, botm = self.botm, top = self.top)
    point = Point(x,y,z)
    icpl = self.vgrid.intersect(x,y)
    pillar = self.vgrid.botm[:,icpl]
    for k in range(self.nlay):
        if z >= pillar[k] and z < pillar[k-1] or z >= pillar[k] and z < self.top[icpl]: # all layers, top layer
            lay = k 
            disu_cell = self.cellid_disu[lay, icpl]
            return(disu_cell)
    else:
        print('z is not within modelgrid')
        
def disucell_layicpl(M, disu_cell): # zero-based
    disv_cell = M.cellid_disv.flatten()[M.cellid_disv.flatten()==disu_cell]  
    
    lay  = math.floor(disv_cell/(M.ncpl)) # Zero based
    icpl = math.floor(disv_cell - lay * M.ncpl) # Zero based
    return(lay, icpl)

C:\Users\19904604\AppData\Local\Temp\ipykernel_3100\3214135597.py:21: DeprecationWarning: GeoPandas is set to use PyGEOS over Shapely. PyGEOS support is deprecatedand will be removed in GeoPandas 1.0, released in the Q1 of 2024. Please migrate to Shapely 2.0 (https://geopandas.org/en/stable/docs/user_guide/pygeos_to_shapely.html).
  import geopandas as gpd


In [ ]:
# angle 1 (DIP DIRECTION) rotates around z axis counterclockwise looking from +ve z.
def find_angle1(nv): # nv = normal vector to surface
    # The dot product of perpencicular vectors = 0
    # A vector perpendicular to nv would be [a,b,c]
    if nv[2] == 0:
        angle1 = 0.
    else:
        a = nv[0]
        b = nv[1]
        c = -(a*nv[0]+b*nv[1])/nv[2]
        v = [a,b,c]
        if np.isnan(v[0]) == True or np.isnan(v[1]) == True: 
            angle1 = 0.
        if v[0] == 0.:
            if v[1] > 0:
                angle1 = 90
            else:
                angle1 = -90
        else:             
            tantheta = v[1]/v[0] 
            angle1 = np.degrees(math.atan(tantheta))
    return(angle1)

# angle 2 (DIP) rotates around y axis clockwise looking from +ve y.
def find_angle2(nv): # nv = normal vector to surface
    # The dot product of perpencicular vectors = 0
    # A vector perpendicular to nv would be [a,b,c]
    if nv[2] == 0:
        angle2 = 0.
    else:
        a = nv[0]
        b = nv[1]
        c = -(a*nv[0]+b*nv[1])/nv[2]
        v = [a,b,c]
        if np.isnan(v[0]) == True or np.isnan(v[1]) == True or np.isnan(v[2]) == True:
            angle2 = 0.
        else:
            v_mag = (v[0]**2 + v[1]**2 + v[2]**2)**0.5 
            costheta = v[2]/v_mag
            angle2 = 90-np.degrees(math.acos(costheta)) 
    return(angle2)

In [ ]:
def evaluate_geo_model(geomodel, xyz):    
    litho = geomodel.evaluate_model(xyz)  # generates an array indicating lithology for every cell
    
    vf1 = geomodel.evaluate_feature_gradient('upper', xyz, scale = True) ## Evaluate gradient field to use for K tensor
    vf2 = geomodel.evaluate_feature_gradient('lower', xyz, scale = True)
    vf1, vf2 = vf1.tolist(), vf2.tolist()

    vf = [None] * len(xyz) # set up an empty list to be able to merge vector fields for form1 and form2
    for i in range(len(xyz)): # There are two vector fields so need to assign which gradient for which layer
        if litho[i] == 0:  vf[i] = vf1[i] 
        if litho[i] == 1:  vf[i] = vf2[i] 
        if litho[i] == 2:  vf[i] = vf2[i] 
        if litho[i] == 3:  vf[i] = vf2[i] 
    return(litho, vf)

In [ ]:
class Reference:
    
    def __init__(self, P):               
        self.modelname = 'Ref'
        self.plan = 'Car'             # Car - cartesian, Tri - triangular, Vor - voronoi
        self.transect = 'Vox'

    def write_arrays(self):    

        modelname = 'Reference'        
        self.nrow, self.ncol, self.nlay = 2*P.nrow, 2*P.ncol, 2*P.nlv
        dz = (P.z1 - P.z0) / self.nlay
        self.dy = (P.y1 - P.y0) / self.nrow
        self.dx = (P.x1 - P.x0) / self.ncol
        delc = np.ones((self.nrow)) * self.dy
        delr = np.ones((self.ncol)) * self.dx
        self.sg = flopy.discretization.StructuredGrid(delc = delc, delr = delr)

        self.top = P.z1 * np.ones((self.nrow, self.ncol), dtype=float)         
        zc = np.arange(P.z0 + dz / 2, P.z1, dz)  # Cell centres
        zbot = np.arange(P.z1 - dz, P.z0 - dz, -dz)
        self.botm = np.zeros((self.nlay, self.nrow, self.ncol)) 
        for lay in range(self.nlay):
            self.botm[lay,:] = zbot[lay]

        xc = np.arange(P.x0 + self.dx/2, P.x1, self.dx) # 26/4
        yc = np.arange(P.y1 - self.dy/2, P.y0, -self.dy)     
      
        xx, yy, zz = np.meshgrid(xc, yc, zc, indexing='ij')
        xyz = np.array([xx.flatten(), yy.flatten(), zz.flatten()]).T
        litho, vf = evaluate_geo_model(P.geomodel, xyz)
        
        def reshape_loop2mf(array):
            array = array.reshape((self.ncol, self.nrow, self.nlay)) # Put into corret array form
            array = array.swapaxes(0,2) # MODFLOW lay, row, col
            array = np.flip(array, 0)   # MODFLOW layers start from top
            return(array)

        angle1, angle2 = [], []
        for i in range(len(vf)):  
            angle1.append(find_angle1(vf[i]))
            angle2.append(find_angle2(vf[i]))

        self.angle1  = reshape_loop2mf(np.asarray(angle1))
        self.angle2  = reshape_loop2mf(np.asarray(angle2))
        self.angle3  = np.zeros_like(angle2) # angle3 always zero
        lith    = reshape_loop2mf(litho)
        
        ############# Added 30/4/24        
        for row in range(self.nrow):
            for col in range(self.ncol):
                point = Point(xc[col], yc[row])
                if P.fault_poly.contains(point):
                    for lay in range(self.nlay):
                        self.angle2[lay,row, col] = 0
        ##############

        self.k11 = np.empty_like(lith, dtype = float) # Create empty 1d arrays
        self.k22 = np.empty_like(lith, dtype = float) # Create empty 1d arrays
        self.k33 = np.empty_like(lith, dtype = float) # Create empty 1d arrays
        self.ss  = np.empty_like(lith, dtype = float) # Create empty 1d arrays

        for i in range(P.nlg):  # replace lithologies with parameters
            self.k11[lith==i] = P.hk[i] 
            self.k22[lith==i] = P.hk[i] 
            self.k33[lith==i] = P.vk[i] 
            self.ss[lith==i]  = P.ss[i]
        self.logk11 = logfunc(self.k11)
        self.logk22 = logfunc(self.k22)
        self.logk33 = logfunc(self.k33)

        self.rch_rec = [] # RECHARGE
        for row in range(self.nrow):
            for col in range(self.ncol): 
                self.rch_rec.append([(0, row, col), P.rch])

        self.chd_rec = [] # HEAD BOUNDARY
        zcentres = zc[::-1] # zc from bottom up, zcentres starts from layer 0
        for row in range(self.nrow):
            col = 0
            x,y = xc[col], yc[row]
            for lay in range(self.nlay):                      
                z = zcentres[lay]
                if zbot[lay] < P.chfunc(x,z):
                    self.chd_rec.append([(lay, row, col), P.chfunc(x,z)])  
        for row in range(self.nrow):    
            col = self.ncol-1
            x,y = xc[col], yc[row]
            for lay in range(self.nlay):                      
                z = zcentres[lay]
                if zbot[lay] < P.chfunc(x,z):
                    self.chd_rec.append([(lay, row, col), P.chfunc(x,z)])    

        self.obslist = []
        def find_lay_pillar(pillar,z):
            lay = 0  # find the layer z is in          
            while lay < self.nlay-1:
                if pillar[lay] > z:
                    lay += 1
                    continue
                else:
                     break
            return (lay)

        for i in range(P.nobs):
            for z in P.zobs:
                x, y = P.xyobsbores[i][0], P.xyobsbores[i][1] # Get coords of obs point
                point = Point(x,y) 
                self.gi = flopy.utils.GridIntersect(self.sg)
                row, col = self.gi.intersect(point)["cellids"][0]
                pillar = self.botm[:,row,col]
                lay = find_lay_pillar(pillar, z)                            
                self.obslist.append([str(str(i) + '_' + str(z)),'head',(lay, row, col)])

        self.spd_wel_past, self.spd_wel_future = [], [] 
        
        for n in range(len(P.xypumpbores)):
            wel_top, wel_bot = P.wel_screens[n][0], P.wel_screens[n][1]
            x, y = P.xypumpbores[n][0], P.xypumpbores[n][1] # Get coords of obs point 
            point = Point(x,y) 
            self.gi = flopy.utils.GridIntersect(self.sg)
            row, col = self.gi.intersect(point)["cellids"][0]
            #wel_col, wel_row, wel_cell_coords = find_cell_dis(P.xypumpbores[n][0], P.xypumpbores[n][1], P.x0, P.y1, self.dx, self.dy)
            nwell_cells = int((wel_top - wel_bot)/dz)
            for lay in range(int((0-wel_top)/dz), int((0-wel_top)/dz) + nwell_cells):           
                self.spd_wel_past.append([(lay, row, col),   P.qwell_past/nwell_cells])
                self.spd_wel_future.append([(lay, row, col), P.qwell_future/nwell_cells])   

    def write_sim(self, period):
    
        sim = flopy.mf6.MFSimulation(sim_name='sim', version='mf6',exe_name=P.mfexe_name, sim_ws=P.workspace)

        if period == 'Steady': tdis = flopy.mf6.modflow.mftdis.ModflowTdis(sim)           
        if period == 'Past':   tdis = flopy.mf6.modflow.mftdis.ModflowTdis(sim, nper=len(P.tdis_past), perioddata=P.tdis_past)
        if period == 'Future': tdis = flopy.mf6.modflow.mftdis.ModflowTdis(sim, nper=len(P.tdis_future), perioddata=P.tdis_future)

        ims = flopy.mf6.ModflowIms(sim, print_option='ALL', complexity = 'Complex',  outer_dvclose = 1e-2, 
                                   inner_dvclose = 1e-3, outer_maximum = 400,)
        gwf = flopy.mf6.ModflowGwf(sim, modelname=self.modelname, save_flows=True)#, newtonoptions = newtonoptions) 
        dis = flopy.mf6.modflow.mfgwfdis.ModflowGwfdis(gwf, nlay=self.nlay, nrow=self.nrow, ncol=self.ncol, 
                                                       delr=self.dx,delc=self.dy,top=self.top, botm=self.botm) 
        npf = flopy.mf6.modflow.mfgwfnpf.ModflowGwfnpf(gwf, xt3doptions=True, k=self.k11, k22=self.k22, k33=self.k33,
                                                       #angle1 = 0., angle2 = 0., angle3 = 0.,
                                                       angle1 = self.angle1, angle2 = self.angle2, angle3 = self.angle3,
                                                       save_flows=False, save_specific_discharge=False,
                                                       icelltype = 1)

        if period == 'Steady': strt = P.strt * np.ones((self.nlay, self.nrow, self.ncol))      
        if period == 'Past':   strt = self.head_ss
        if period == 'Future': strt = self.head_present

        ic = flopy.mf6.ModflowGwfic(gwf, strt = strt)
        ch = flopy.mf6.modflow.mfgwfchd.ModflowGwfchd(gwf, maxbound=len(self.chd_rec),stress_period_data=self.chd_rec)
        rch = flopy.mf6.modflow.mfgwfrch.ModflowGwfrch(gwf, maxbound=len(self.rch_rec),stress_period_data=self.rch_rec)

        if period == 'Past': 
            sto = flopy.mf6.modflow.mfgwfsto.ModflowGwfsto(gwf, ss=self.ss, sy=0.1, iconvert=1)
            wel = flopy.mf6.modflow.mfgwfwel.ModflowGwfwel(gwf, stress_period_data = self.spd_wel_past)

        if period == 'Future': 
            sto = flopy.mf6.modflow.mfgwfsto.ModflowGwfsto(gwf, ss=self.ss, sy=0.1, iconvert=1)
            wel = flopy.mf6.modflow.mfgwfwel.ModflowGwfwel(gwf, stress_period_data = self.spd_wel_future)

        if period == 'Steady': csv_file = self.modelname + "_steady.csv"    
        if period == 'Past':   csv_file = self.modelname + "_past.csv"
        if period == 'Future': csv_file = self.modelname + "_future.csv" 

        oc = flopy.mf6.ModflowGwfoc(gwf, budget_filerecord='{}.bud'.format(self.modelname), 
                                    head_filerecord='{}.hds'.format(self.modelname), saverecord=[('HEAD', 'LAST'),])
        obsdict = {csv_file: self.obslist}
        obs = flopy.mf6.ModflowUtlobs(gwf, filename=self.modelname, print_input=True, continuous=obsdict)
        sim.write_simulation(silent = True)  
        print('Reference Sim Written for ', period) 
        return(sim)
    
    def run_sim(self, sim):
        start = datetime.now()
        success, buff = sim.run_simulation(silent = True)   
        print('Model success = ', success)

        if success:
            fname = '{}.hds'.format(self.modelname)
            hds = flopy.utils.binaryfile.HeadFile(os.path.join(P.workspace, fname))  
            times = hds.get_times()
            head = hds.get_data(totim=times[-1]) 
            gwf = sim.get_model() 
            self.gwf = gwf
            obs_data = gwf.obs
            end = datetime.now()
            runtime = (end - start).total_seconds()
            print('run_time = ', runtime, '\n')
            return(head, obs_data, runtime)   

In [ ]:
class Model:
    
    def __init__(self,
                 modelname,
                 P,                # Project parameters
                 plan,             # plan (Car - cartesian, Tri - triangular, Vor - voronoi), 
                 transect,):       # transect (Vox - voxel, Con - conformable)
    
        #---------------- CREATE PLAN DISCRETISATION ARRAYS ------------------------------------#
        
        self.modelname = modelname             # e.g. 'REF', 'SS', 'SU'...
        self.plan = plan             # Car - cartesian, Tri - triangular, Vor - voronoi
        self.transect = transect     # Vox - voxel, Con - conformable
           
        # Retrieve these details from meshing
        if plan == 'car':
            self.cell2d   = P.cell2dcar
            self.ncpl     = len(self.cell2d)
            self.vertices = P.verticescar
            self.xcyc     = P.xcyccar
            
        if plan == 'tri':
            self.cell2d   = P.cell2dtri
            self.ncpl     = len(self.cell2d)
            self.vertices = P.verticestri
            self.xcyc     = P.xcyctri
            
        if plan == 'vor':
            self.cell2d   = P.cell2dvor
            self.ncpl     = len(self.cell2d)
            self.vertices = P.verticesvor
            self.xcyc     = P.xcycvor
            
#---------- FUNCTION TO EVALUATE GEO MODEL AND POPULATE HYDRAULIC PARAMETERS ------#
# LOOP2MF take a flexible grid in plan and regular grid in transect and evaluates geo model    

    def create_lith_dis_arrays(self, P): # Takes the project parameters and model class. 
        
        print('Creating lithology and discretisation arrays for ', self.modelname, ' ...')
        t0 = datetime.now()
        
        def reshape_loop2mf(array):
            array = array.reshape((self.nlay, self.ncpl))
            array = np.flip(array, 0)
            return(array)

#---------- VOX - DIS ARRAY ------#

        if self.transect == 'vox':
            
            self.nlay = P.nlv
            self.nlm = P.nlv
            self.dz = (P.z1 - P.z0) / P.nlv
            self.ncell3d = self.ncpl * self.nlay
            self.idomain = np.ones((self.nlay, self.ncpl)) 
            self.top = P.z1 * np.ones((self.ncpl), dtype=float)
            
            self.zc = np.arange(P.z0 + self.dz / 2, P.z1, self.dz)  # Cell centres
            self.zbot = np.arange(P.z1 - self.dz, P.z0 - self.dz, -self.dz)
            
            self.botm = np.zeros((self.nlay, self.ncpl)) 
            for lay in range(self.nlay):
                self.botm[lay,:] = self.zbot[lay]

            #----- VOX - LITH AND VF ------#

            xyz = []                         
            for k in range(self.nlay):
                z = self.zc[k]
                for i in range(self.ncpl):    
                    x, y = self.xcyc[i][0], self.xcyc[i][1]
                    xyz.append((x,y,z))
            
            # Reshape to lay, ncpl
            #litho, vf = evaluate_geo_model(P.geomodel, xyz)
            litho = P.geomodel.evaluate_model(xyz)
            print(np.unique(litho, return_counts=True))
            
            litho = np.asarray(litho)
            litho = litho.reshape((self.nlay, self.ncpl))
            litho = np.flip(litho, 0)
            self.lith = litho
            
            #angle1, angle2 = [], []
            #for i in range(len(vf)):  
            #    angle1.append(find_angle1(vf[i]))
            #    angle2.append(find_angle2(vf[i]))
            #self.angle1  = reshape_loop2mf(np.asarray(angle1))
            #self.angle2  = reshape_loop2mf(np.asarray(angle2))
            
#---------- CON - EVALUATE LITH ON FINE REGULAR LAYERS ------#

        if self.transect == 'con': # CREATING DIS AND NPF ARRAYS
            
            res = 2 # 2m resolution
            nlay = int((P.z1 - P.z0)/res)
            dz = (P.z1 - P.z0)/nlay # actual resolution
            zc = np.arange(P.z0 + dz / 2, P.z1, dz)  # Cell centres
            
            xyz = []                         
            for k in range(nlay):
                z = zc[k]
                for i in range(self.ncpl):    
                    x, y = self.xcyc[i][0], self.xcyc[i][1]
                    xyz.append((x,y,z))
            litho = P.geomodel.evaluate_model(xyz)
            #litho, vf = evaluate_geo_model(P.geomodel, xyz) # We won't be using this vf. Will re-evaluate gradient later
            litho = np.asarray(litho)
            litho = litho.reshape((nlay, self.ncpl)) # Reshape to lay, ncpl
            litho = np.flip(litho, 0)
            
            #----- CON - CREATE LITH, BOTM AND IDOMAIN ARRAYS (PILLAR METHOD, PICKS UP PINCHED OUT LAYERS) ------#

            def start_stop_arr(initial_list): # Function to look down pillar and pick geo bottoms
                a = np.asarray(initial_list)
                mask = np.concatenate(([True], a[1:] != a[:-1], [True]))
                idx = np.flatnonzero(mask)
                l = np.diff(idx)
                start = np.repeat(idx[:-1], l)
                stop = np.repeat(idx[1:]-1, l)
                return(start, stop)
            
            self.nlm    = P.nlg * P.nls # number of model layers = geo layers * sublayers 
            top         = P.z1 * np.ones((self.ncpl), dtype=float)
            botm_geo    = np.zeros((P.nlg, self.ncpl), dtype=float) # bottom elevation of each geological layer
            botm        = np.zeros((self.nlm, self.ncpl), dtype=float) # bottom elevation of each model layer
            idomain_geo = np.zeros_like(botm_geo, dtype=int)      # idomain array for each lithology
            idomain     = np.ones((self.nlm, self.ncpl), dtype=int)    # idomain for each model layer

            for icpl in range(self.ncpl): 
                #get strat column
                strat_log = litho[:,icpl]
                present = np.unique(strat_log)
                start, stop =  start_stop_arr(strat_log)
                start = np.unique(start)
                stop = np.unique(stop)
                for i, lith in enumerate(present):           
                    idomain_geo[lith, icpl] = 1
                    botm_geo[lith, icpl] = P.z1 - (stop[i]+1) * dz
                for lay_geo in range(P.nlg):
                    for lay_sub in range(P.nls):
                        lay = lay_geo * P.nls + lay_sub
                        if idomain_geo[lay_geo, icpl] == 0: # if pinched out geological layer...
                            idomain[lay, icpl] = 0          # model cell idomain = 0
                            botm_geo[lay_geo, icpl] = botm_geo[lay_geo-1, icpl]  

            # Creates bottom of model layers
            lay_geo = 0 # Start with top geological layer
            botm[0,:] = top - (top - botm_geo[0])/P.nls # Very first model layer
            
            for i in range(1, P.nls): # First geo layer. i represent sublay 0,1,2 top down within layer
                lay = lay_geo * P.nls + i
                botm[lay,:] = top - (i+1) * (top - botm_geo[0])/P.nls

            for lay_geo in range(1, P.nlg): # Work through subsequent geological layers
                for i in range(P.nls): 
                    lay = lay_geo * P.nls + i
                    botm[lay,:] = botm_geo[lay_geo-1] - (i+1) * (botm_geo[lay_geo-1] - botm_geo[lay_geo])/P.nls
       
            self.botm = botm
            self.top = top
            self.idomain = idomain
            self.nlay = P.nlg * P.nls
            
            self.lith  = np.ones_like(self.botm, dtype = float)
            for lay_geo in range(P.nlg):
                for i in range(P.nls):
                    lay = lay_geo * P.nls + i 
                    self.lith[lay,:] *= lay_geo
                    
            # Get gradients by reevaluationg model at bottoms
            
            xyz = []                         
            for lay in range(self.nlay-1, -1, -1):
                for icpl in range(self.ncpl):  
                    x, y, z = self.xcyc[icpl][0], self.xcyc[icpl][1], self.botm[lay, icpl] 
                    xyz.append((x,y,z))
            #litho, vf = evaluate_geo_model(P.geomodel, xyz)
            litho = P.geomodel.evaluate_model(xyz)
            #angle1, angle2 = [], []
            #for i in range(len(vf)):  
            #    angle1.append(find_angle1(vf[i]))
            #    angle2.append(find_angle2(vf[i]))
            #self.angle1  = reshape_loop2mf(np.asarray(angle1))
            #self.angle2  = reshape_loop2mf(np.asarray(angle2))
        
        t1 = datetime.now()
        run_time = t1 - t0
        print('Time taken = ', run_time.total_seconds())
        
# ------------------ MAKE MODEL GRID CLASS-----------------#
        self.nnodes_div = len(self.botm.flatten())    
        self.vgrid = flopy.discretization.VertexGrid(vertices=self.vertices,
                               cell2d=self.cell2d,
                               top = self.top,
                               idomain=self.idomain,
                               botm=self.botm,
                               nlay=self.nlm, #number of layers in model
                               ncpl=self.ncpl,)

################## PROP ARRAYS TO BE SAVED IN DISU FORMAT ##################        
    def create_prop_arrays(self, P): # Uses lithology codes to populate arrays 
        
        print('Creating property arrays for ', self.modelname, ' ...')
        t0 = datetime.now()
        
        # First create an array for cellids in layered version  (before we pop cells that are absent)
        self.cellid_disv = np.empty_like(self.lith, dtype = int)
        self.cellid_disu = -1 * np.ones_like(self.lith, dtype = int)
        i = 0
        for lay in range(self.nlay):
            for icpl in range(self.ncpl):
                cellid = lay * self.ncpl + icpl
                self.cellid_disv[lay, icpl] = cellid
                if self.idomain[lay, icpl] != 0:
                    self.cellid_disu[lay, icpl] = i
                    i += 1
        self.ncell_disv = len(self.cellid_disv.flatten())
        self.ncell_disu = np.count_nonzero(self.cellid_disu >= 0)
        
#---------- PROP ARRAYS (VOX and CON) -----   
        self.k11    = np.empty_like(self.lith, dtype = float)
        self.k22    = np.empty_like(self.lith, dtype = float)
        self.k33    = np.empty_like(self.lith, dtype = float)
        self.ss     = np.empty_like(self.lith, dtype = float)

        for n in range(P.nlg):  # replace lithologies with parameters
            self.k11[self.lith==n] = P.hk[n] 
            self.k22[self.lith==n] = P.hk[n] 
            self.k33[self.lith==n] = P.vk[n] 
            self.ss[self.lith==n]  = P.ss[n]
            
        ## ADDED 30/4
        print(self.angle2.shape)
        
        # Force all K tensor angles in fault zone to 0 (Loop can't calculate angles in faulted area properly yet!)
        '''for icpl in range(self.ncpl):
            point = Point(self.xcyc[icpl])
            if P.fault_poly.contains(point):
                for lay in range(self.nlay):
                    self.angle2[lay,icpl] = 0 '''     
        ######################################
        
        self.k11    = self.k11[self.cellid_disu >-1].flatten()
        self.k22    = self.k22[self.cellid_disu >-1].flatten()
        self.k33    = self.k33[self.cellid_disu >-1].flatten()
        self.ss     = self.ss[self.cellid_disu >-1].flatten()
        #self.angle1 = self.angle1[self.cellid_disu >-1].flatten()
        #self.angle2 = self.angle2[self.cellid_disu >-1].flatten()
        #self.angle3 = np.zeros_like(self.angle1, dtype = float)  # Angle 3 always at 0
        
        self.logk11    = logfunc(self.k11)
        self.logk22    = logfunc(self.k22)
        self.logk33    = logfunc(self.k33)
        
        t1 = datetime.now()
        run_time = t1 - t0
        print('Time taken = ', run_time.total_seconds())

################## FLOW PACKAGES TO BE SAVED IN DISU FORMAT ##################    
    def create_flow_package_arrays(self, P, rch = True, chd = True, obs = True, wel = True): # e.g. SS.add_flow_packages(recharge = True, chd = True)        
            
        t0 = datetime.now()
        Car = flopy.discretization.VertexGrid(vertices = self.vertices, cell2d = self.cell2d, top = self.top)
        
        print('Adding flow packages to ', self.modelname, ' ...')
        
        self.strt = P.strt * np.ones_like(self.k11, dtype = float)
        
        if rch:
            self.rch_rec = [] # RECHARGE
            for icpl in range(self.ncpl): 
                self.rch_rec.append([(icpl,), P.rch])
                
        if chd:
            self.vgrid = flopy.discretization.VertexGrid(ncpl = self.ncpl, vertices = self.vertices, cell2d = self.cell2d,
                                                     nlay = self.nlm, botm = self.botm, top = self.top)
            self.gi = flopy.utils.GridIntersect(self.vgrid)
            
            west_bd = LineString([(P.x0, P.y0), (P.x0, P.y1)]) # Western edge
            west_cells = self.gi.intersects(west_bd, shapetype="linestring")
            west_cells = west_cells.cellids.tolist()
                     
            east_bd = LineString([(P.x1, P.y0), (P.x1, P.y1)]) # Eastern edge
            east_cells = self.gi.intersects(east_bd, shapetype="linestring")
            east_cells = east_cells.cellids.tolist()
            
            self.chd_rec = [] # HEAD BOUNDARY
            for icpl in west_cells:
                x,y = self.xcyc[icpl][0], self.xcyc[icpl][1]
                for lay in range(self.nlay):
                    z = lay_to_z(self.botm, self.top, lay, icpl=icpl)
                    zb = self.botm[lay,icpl]
                    if zb < P.chfunc(x,z):                           
                        cell_disv = icpl + lay*(self.ncpl) #8/5
                        cell_disu = self.cellid_disu.flatten()[cell_disv]
                        if cell_disu != -1:
                            self.chd_rec.append([cell_disu, P.chfunc(x,z)]) 
                            
            for icpl in east_cells:
                x,y = self.xcyc[icpl][0], self.xcyc[icpl][1]
                for lay in range(self.nlay):
                    z = lay_to_z(self.botm, self.top, lay, icpl=icpl)
                    zb = self.botm[lay,icpl]
                    if zb < P.chfunc(x,z):                           
                        cell_disv = icpl + lay*(self.ncpl)
                        cell_disu = self.cellid_disu.flatten()[cell_disv]
                        if cell_disu != -1:
                            self.chd_rec.append([cell_disu, P.chfunc(x,z)]) 
        if obs:        
            self.obslist = []
            
            def find_lay_pillar(pillar,z):
                lay = 0  # find the layer z is in          
                while lay < self.nlay-1:
                    if pillar[lay] > z:
                        lay += 1
                        continue
                    else:
                         break
                return (lay)       
            
            for i in range(P.nobs): # Plan
                for z in P.zobs:    # Depth
                    x, y = P.xyobsbores[i][0], P.xyobsbores[i][1] # Get coords of obs point
                    point = Point(x,y) 
                    self.gi = flopy.utils.GridIntersect(self.vgrid)
                    icpl = self.gi.intersect(point)["cellids"]
                    icpl = np.array(list(icpl)) 
                    icpl = icpl[0]

                    pillar = self.botm[:,icpl]
                    lay = find_lay_pillar(pillar,z)  
                    cell_disv = icpl + lay*self.ncpl 
                    cell_disu = self.cellid_disu.flatten()[cell_disv]
                    self.obslist.append([str(P.idobsbores[i] + '_' + str(z)),'head',(cell_disu+1)])

        if wel:
            self.spd_wel_past, self.spd_wel_future = [], [] 

            for n in range(len(P.xypumpbores)):
                x, y = P.xypumpbores[n][0], P.xypumpbores[n][1] # Get coords of obs point
                point = Point(x,y) 
                self.gi = flopy.utils.GridIntersect(self.vgrid)
                icpl = self.gi.intersect(point)["cellids"]
                icpl = np.array(list(icpl)) 
                icpl = icpl[0]
                
                #wel_icpl, wel_icplcoords = find_cell_disv(wel_coords[0], wel_coords[1], self.xcyc)
                wel_top, wel_bot = P.wel_screens[n][0], P.wel_screens[n][1]     
                
                if self.transect == 'vox':
                    nwell_cells = int((wel_top - wel_bot)/self.dz)
                    print(n, nwell_cells)
                    for lay in range(int((0-wel_top)/self.dz), int((0-wel_top)/self.dz) + nwell_cells):   
                        cell_disv = icpl + lay*self.ncpl
                        cell_disu = self.cellid_disu.flatten()[cell_disv]
                        self.spd_wel_past.append([cell_disu, P.qwell_past/nwell_cells])
                        self.spd_wel_future.append([cell_disu, P.qwell_future/nwell_cells])

                if self.transect == 'con':        
                    nwell_cells = P.nls # For this research, assume pumping across entire geological layer
                    for wel_lay in range(P.geo_pl * P.nls, (P.geo_pl + 1) * P.nls): # P.geo_pl = geological pumped layer                    
                        cell_disv = icpl + wel_lay*self.ncpl
                        cell_disu = self.cellid_disu.flatten()[cell_disv]
                        self.spd_wel_past.append([cell_disu, P.qwell_past/nwell_cells])
                        self.spd_wel_future.append([cell_disu, P.qwell_future/nwell_cells])
                        
        t1 = datetime.now()
        run_time = t1 - t0
        print('Time taken = ', run_time.total_seconds())
                
    def write_run_model(self, P, period, complexity='Complex', outer_dvclose=1e-2, inner_dvclose=1e-3, 
                                   outer_maximum=200, newtonoptions = False):
        
        print('Writing simulation and gwf for ', self.modelname, ' ...')
        print(self.modelname)
        t0 = datetime.now()
        
        # -------------- SIM -------------------------
        sim = flopy.mf6.MFSimulation(sim_name='sim', version='mf6',exe_name=P.mfexe_name, sim_ws=P.workspace)

        # -------------- TDIS -------------------------
        if period == 'Steady': tdis = flopy.mf6.modflow.mftdis.ModflowTdis(sim)           
        if period == 'Past':   tdis = flopy.mf6.modflow.mftdis.ModflowTdis(sim, nper=len(P.tdis_past), perioddata=P.tdis_past)
        if period == 'Future': tdis = flopy.mf6.modflow.mftdis.ModflowTdis(sim, nper=len(P.tdis_future), perioddata=P.tdis_future)
        
        # -------------- IMS -------------------------
        ims = flopy.mf6.ModflowIms(sim, print_option='ALL', 
                                   complexity    = 'Complex',
                                   outer_dvclose = 1e-2, 
                                   inner_dvclose = 1e-3, 
                                   outer_maximum = 400,)
                                   #inner_maximum = 500,
                                   #linear_acceleration = "BICGSTAB"
                                   #reordering_method=['RCM']
                                   #under_relaxation_gamma = 0.6, #No real impact
                                   #under_relaxation_kappa = 1e-7, #Making this lower helps to gte convergence
                                   #under_relaxation_theta = 0.2,
                                   #under_relaxation_momentum = 0.2,

        # -------------- GWF -------------------------
        gwf = flopy.mf6.ModflowGwf(sim, modelname=self.modelname, save_flows=True, newtonoptions = newtonoptions,) 

        # -------------- DIS -------------------------       

        import disv2disu_JM
        from importlib import reload
        reload(disv2disu_JM)
        Disv2Disu = disv2disu_JM.Disv2Disu           

        if self.transect == 'vox':
            staggered = False # Layered connectivity
        
        if self.transect == 'con':
            staggered = True # Full connectivity
            
        dv2d = Disv2Disu(self.vertices, self.cell2d, self.top, self.botm, staggered=staggered, disv_idomain = self.idomain)
        disu_gridprops = dv2d.get_gridprops_disu6()
        disu = flopy.mf6.ModflowGwfdisu(gwf, **disu_gridprops) # This is the flow package
        
        # -------------- NPF -------------------------
               
        npf = flopy.mf6.modflow.mfgwfnpf.ModflowGwfnpf(gwf, xt3doptions=True, k=self.k11, k22=self.k22, k33=self.k33, 
                                                       #angle1 = self.angle1, angle2 = self.angle2, angle3 = self.angle3, 
                                                       angle1 = 0., angle2 = 0., angle3 = 0.,
                                                       icelltype = 1,
                                                       save_flows=False, save_specific_discharge=False,)
                                                       #dev_minimum_saturated_thickness = 1)# try 0.1 then 0.001... no more than 1m!
        
        # -------------- IC / STO / WEL-------------------------
        if period == 'Steady': 
            ic = flopy.mf6.ModflowGwfic(gwf, strt = self.strt)
            csv_file = self.modelname + "_steady.csv"   #To write observation to  
        if period == 'Past':   
            ic = flopy.mf6.ModflowGwfic(gwf, strt = self.head_ss,)    
            sto = flopy.mf6.modflow.mfgwfsto.ModflowGwfsto(gwf, storagecoefficient=None, iconvert=1, 
                                                           ss=self.ss, sy = 0.1,)
            wel = flopy.mf6.modflow.mfgwfwel.ModflowGwfwel(gwf, print_input=True, print_flows=True, 
                                                           stress_period_data = self.spd_wel_past, 
                                                           save_flows=True,)
            csv_file = self.modelname + "_past.csv"   #To write observation to      
        if period == 'Future':
            ic = flopy.mf6.ModflowGwfic(gwf, strt = self.head_present,)
            sto = flopy.mf6.modflow.mfgwfsto.ModflowGwfsto(gwf, storagecoefficient=None, iconvert=1, 
                                                           ss=self.ss, sy = 0.1,)
            wel = flopy.mf6.modflow.mfgwfwel.ModflowGwfwel(gwf, print_input=True, print_flows=True, 
                                                           stress_period_data = self.spd_wel_future, 
                                                           save_flows=True,)
            csv_file = self.modelname + "_future.csv" # To write observation to
            
        # -------------- CH / RCH -------------------------
        ch = flopy.mf6.modflow.mfgwfchd.ModflowGwfchd(gwf, maxbound=len(self.chd_rec),stress_period_data=self.chd_rec,)
        rch = flopy.mf6.modflow.mfgwfrch.ModflowGwfrch(gwf, maxbound=len(self.rch_rec),stress_period_data=self.rch_rec,)
               
        # -------------- OBS / OC -------------------------
        obsdict = {csv_file: self.obslist}
        obs = flopy.mf6.ModflowUtlobs(gwf, filename=self.modelname, print_input=True, continuous=obsdict,)

        oc = flopy.mf6.ModflowGwfoc(gwf, budget_filerecord='{}.bud'.format(self.modelname), 
                                    head_filerecord='{}.hds'.format(self.modelname),
                                    saverecord=[('HEAD', 'LAST'),], printrecord=None,)
        
        # -------------- WRITE AND RUN SIMULATION -------------------------
        sim.write_simulation(silent = True)   

        print('Running simulation for ', self.modelname, ' ...')
        success, buff = sim.run_simulation(silent = True)   
        print('Period = ', period, '\nModel success = ', success)

        if success:
            fname = '{}.hds'.format(self.modelname)
            hds = flopy.utils.binaryfile.HeadFile(os.path.join(P.workspace, fname))  
            times = hds.get_times()
            head = hds.get_data(totim=times[-1]) 
            #bdobj = gwf.output.budget()
            #spdis = bdobj.get_data(text="DATA-SPDIS")[0]
            obs_data = gwf.obs
            t1 = datetime.now()
            run_time = t1 - t0
            print('run_time = ', run_time.total_seconds())
            return(gwf, head, obs_data, run_time.total_seconds())           


In [ ]:
def run_prediction(M, param_set): # hk, vk list of 4 long
    
    hk0, kh1, hk2, hk3, vk0, vk1, vk2, vk3, ss0, ss1, ss2, ss3 = param_set
    hk, vk = [10**hk0, 10**kh1, 10**hk2, 10**hk3], [10**vk0, 10**vk1, 10**vk2, 10**vk3]
    ss = [10**ss0, 10**ss1, 10**ss2, 10**ss3]
    M.add_well(qwell_future)
    gwf, head, obs_data, run_time = M.write_run_model(workspace, period = 'Future') 
    M.pred_head = np.copy(head)
    csv_file = '../modelfiles/' + M.modelname + '_future.csv'
    data_set = pd.read_csv(csv_file, header=1)
    data_frames = pd.DataFrame(data_set)
    hobs = np.array(data_frames.values)
    hobs = hobs[:,1::]
    hobs = np.swapaxes(hobs, 0, 1)
    
    return(hobs)

In [ ]:
def update_pars(hk0, hk1, hk2, hk3, vk0, vk1, vk2, vk3, ss0, ss1, ss2, ss3):
    
    print('running update_pars')
    hk, vk = [10**hk0, 10**hk1, 10**hk2, 10**hk3], [10**vk0, 10**vk1, 10**vk2, 10**vk3]
    ss = [10**ss0, 10**ss1, 10**ss2, 10**ss3]
    
          
    if M.gridtype == 'UU':
        M.k11 = np.ones((M.nlay_mod, M.ncpl))
        M.k22 = np.ones((M.nlay_mod, M.ncpl))
        M.k33 = np.ones((M.nlay_mod, M.ncpl))
        M.ss  = np.ones((M.nlay_mod, M.ncpl))

        for geolay in range(nlay_geo):
            for i in range(M.nlay_sub):
                lay = geolay * M.nlay_sub + i
                M.k11[lay,:] *= hk[geolay]
                M.k22[lay,:] *= hk[geolay]
                M.k33[lay,:] *= vk[geolay]
                M.ss[lay,:]  *= ss[geolay]
            
        M.k11 = M.k11.flatten()
        M.k22 = M.k22.flatten()
        M.k33 = M.k33.flatten()
        M.ss  = M.ss.flatten()

def model_make_n_run(hk0, hk1, hk2, hk3, vk0, vk1, vk2, vk3, ss0, ss1, ss2, ss3): # hk, vk list of 4 long
    #start = datetime.now()
    update_pars(hk0, hk1, hk2, hk3, vk0, vk1, vk2, vk3, ss0, ss1, ss2, ss3)

    M.gwf, M.ss_head, M.ss_obs, run_time = M.write_run_model(workspace, period = 'Steady') # Run Steady State

    M.add_well(qwell_past) # Run Transient
    M.gwf, M.head_present, M.obs_present, run_time = M.write_run_model(workspace, period = 'Past') 
    
    fname = modelname + "_past.csv"
    csv_file = os.path.join(workspace, fname)
    data_set = pd.read_csv(csv_file, header=1)
    data_frames = pd.DataFrame(data_set)
    hobs = np.array(data_frames.values)
    hobs = hobs[:,1:]
    hobs = np.swapaxes(hobs, 0, 1)
    #print(np.unique(hobs))
    #end = datetime.now()
    #run_time = end - start
    return(hobs, run_time)

In [ ]:
def test_complexity():
    # Run Steady state, Past, Future
    models = [SS, US, SU, UU]
    nruns = 5
    complex_options = ['Moderate', 'Complex']

    run_time_results = np.zeros((len(models), 3, len(complex_options), nruns))

    for m, M in enumerate(models): 
        for c in range(len(complex_options)):
            for n in range(nruns):

                M.gwf, M.ss_head, M.ss_obs, run_time = M.write_run_model(workspace, period = 'Steady', 
                                                                         complexity = complex_options[c]) 
                run_time_results[m, 0, c, n] = run_time

                M.add_well(wel_coords, wel_bot, wel_top, qwell_past)
                M.gwf, M.head_present, M.obs_present, run_time = M.write_run_model(workspace, period = 'Past',
                                                                                  complexity = complex_options[c]) 
                run_time_results[m, 1, c, n] = run_time

                M.add_well(wel_coords, wel_bot, wel_top, qwell_future)
                M.gwf, M.head_future, M.obs_future, run_time = M.write_run_model(workspace, period = 'Future',
                                                                                complexity = complex_options[c]) 
                run_time_results[m, 2, c, n] = run_time

In [ ]:
class Project:
    
    def __init__(self, 
                 projectname, # string
                 boundingbox, # for geo model: tuple(x0, x1, y0, y1, z0, z1)
                 ):
        
        self.projectname = projectname
        self.boundingbox = boundingbox
        self.x0, self.x1 = boundingbox[0], boundingbox[1]
        self.y0, self.y1 = boundingbox[2], boundingbox[3]
        self.z0, self.z1 = boundingbox[4], boundingbox[5]
        self.Lx = self.x1 - self.x0
        self.Ly = self.y1 - self.y0
        self.Lz = self.z1 - self.z0

In [ ]:
def prepboundarymesh(P, grid): # MODEL BOUNDARY
    
    x0, x1, y0, y1 = P.x0, P.x1, P.y0, P.y1 
    w, r  = P.w, P.r
    Lx, Ly = x1 - x0, y1 - y0
    
    if grid == 'tri':
        model_vertices = []
        for i in range(r): model_vertices.append(((i/r * Lx)+x0,y0))         # Bottom
        for i in range(r): model_vertices.append((x1, (i/r * Ly) + y0))      # Right
        for i in range(r): model_vertices.append(((Lx - i/r * Lx) + x0, y1)) # Top
        for i in range(r): model_vertices.append((x0, (Ly - i/r * Ly) +y0))  # Left

        # INTERIOR BOUNDARY
        interior_vertices = [(x0+w,y0+w),(x1-w,y0+w),(x1-w, y1-w),(x0+w,y1-w)]
        interior_poly = Polygon(interior_vertices)
        
    if grid == 'vor':
        model_vertices = []
        for i in range(r): model_vertices.append(((i/r * Lx)+x0,y0))         # Bottom
        for i in range(r): model_vertices.append((x1, (i/r * Ly) + y0))      # Right
        for i in range(r): model_vertices.append(((Lx - i/r * Lx) + x0, y1)) # Top
        for i in range(r): model_vertices.append((x0, (Ly - i/r * Ly) +y0))  # Left

        # INTERIOR BOUNDARY
        interior_vertices = [(x0+w,y0+w),(x1-w,y0+w),(x1-w, y1-w),(x0+w,y1-w)]
        interior_poly = Polygon(interior_vertices)
    
    return(model_vertices, interior_vertices)

def prepboremesh(P, grid):
    
    theta = np.linspace(0, 2 * np.pi, 11)

    pump_bores_inner, pump_bores_outer = [], []
    obs_bores_inner, obs_bores_outer = [], []
    
    if grid == 'tri':
        
        def vertices_equtri1(X, Y, l): # l is distance from centre of triangle to vertex
            x1 = X - l*3**0.5/2 
            x2 = X + l*3**0.5/2 
            x3 = X
            y1 = Y - l/2
            y2 = Y - l/2
            y3 = Y + l
            return(x1, x2, x3, y1, y2, y3)
        
        def vertices_equtri2(X, Y, l): # l is distance from centre of triangle to vertex
            x1 = X 
            x2 = X + l*3**0.5
            x3 = X - l*3**0.5
            y1 = Y - 2*l
            y2 = Y + l
            y3 = Y + l
            return(x1, x2, x3, y1, y2, y3)

        for i in P.xypumpbores:   
            X, Y = i[0], i[1] # coord of pumping bore
                        
            x1, x2, x3, y1, y2, y3 = vertices_equtri1(X, Y, P.radius1/2)
            vertices_inner = ((x1, y1), (x2, y2), (x3, y3))
            x1, x2, x3, y1, y2, y3 = vertices_equtri2(X, Y, P.radius1/2)
            vertices_outer = ((x1, y1), (x2, y2), (x3, y3))
            
            pump_bores_inner.append(vertices_inner)
            pump_bores_outer.append(vertices_outer)
            
        for i in P.xyobsbores:   
            X, Y = i[0], i[1] # coord of pumping bore
                        
            x1, x2, x3, y1, y2, y3 = vertices_equtri1(X, Y, P.radius1/2)
            vertices_inner = ((x1, y1), (x2, y2), (x3, y3))
            x1, x2, x3, y1, y2, y3 = vertices_equtri2(X, Y, P.radius1/2)
            vertices_outer = ((x1, y1), (x2, y2), (x3, y3))
            
            obs_bores_inner.append(vertices_inner)
            obs_bores_outer.append(vertices_outer)
            
        return(pump_bores_inner, pump_bores_outer, obs_bores_inner, obs_bores_outer)
    
    if grid == 'vor':
        for i in P.xypumpbores:   
            X = i[0] + P.radius1 * np.cos(theta)
            Y = i[1] + P.radius1 * np.sin(theta)    
            vertices_inner = [(x_val, y_val) for x_val, y_val in zip(X, Y)]
            X = i[0] + P.radius2 * np.cos(theta)
            Y = i[1] + P.radius2 * np.sin(theta)    
            vertices_outer = [(x_val, y_val) for x_val, y_val in zip(X, Y)]
            pump_bores_inner.append(vertices_inner)
            pump_bores_outer.append(vertices_outer)
            
        for i in P.xyobsbores:   
            X = i[0] + P.radius1 * np.cos(theta)
            Y = i[1] + P.radius1 * np.sin(theta)    
            vertices_inner = [(x_val, y_val) for x_val, y_val in zip(X, Y)]
            X = i[0] + P.radius2 * np.cos(theta)
            Y = i[1] + P.radius2 * np.sin(theta)    
            vertices_outer = [(x_val, y_val) for x_val, y_val in zip(X, Y)]
            obs_bores_inner.append(vertices_inner)
            obs_bores_outer.append(vertices_outer)

        return(pump_bores_inner, pump_bores_outer, obs_bores_inner, obs_bores_outer)

def prepfaultmesh(P, grid):
    
    L = P.fault_buffer
    Lfault = np.sqrt((P.fx2-P.fx1)**2+(P.fy2 - P.fy1)**2)
    from shapely.geometry import LineString,Point,Polygon,shape
         # refining factor - adds points along fault for triangulation
    fs = (P.fy2 - P.fy1)/(P.fx2 - P.fx1) # fault strike in xy
    fp = -1/fs # direction perpendiculr to fault strike
    theta = math.atan(fp)
    phi = math.pi/2 - theta
    
    K = P.fault_buffer
    fp1 = (P.fx1 + K * np.cos(phi), P.fy1 + K * np.sin(phi))
    fp2 = (P.fx2 + K * np.cos(phi), P.fy2 + K * np.sin(phi))
    fp3 = (P.fx2 - K * np.cos(phi), P.fy2 - K * np.sin(phi))
    fp4 = (P.fx1 - K * np.cos(phi), P.fy1 - K * np.sin(phi))
    P.fault_poly = Polygon((fp1, fp2, fp3, fp4))
    
    if grid == 'tri':
        
        # THESE WILL BE NODES
        r = 50      # refining factor
        fault_points = []
        for i in range(r+1): 
            fault_points.append((P.fx1 + i * (P.fx2-P.fx1)/r , P.fy1 + i * (P.fy2-P.fy1)/r)) 
    
    if grid == 'vor':

        r = 2*L/3 # distance between points

        x_array = np.arange(0, Lfault, r)  # Create a cloud of points to refine around fault
        y_array = np.arange(-L, L + r, r)

        fault_points = []
        for i in range(len(x_array)):# vertical points 
            for j in range(len(y_array)): # horizontal points
                x = x_array[i] * np.cos(phi) + y_array[j] * np.sin(phi)
                y = x_array[i] * -np.sin(phi) + y_array[j] * np.cos(phi)
                fault_points.append((P.fx1 + x, P.fy1 + y))
            
    return(fault_points)

def createcell2d(P, grid, fault = True):
    
    if grid == 'car':
        delr = P.delx * np.ones(P.ncol, dtype=float)
        delc = P.dely * np.ones(P.nrow, dtype=float)
        top  = np.ones((P.nrow, P.ncol), dtype=float)
        botm = np.zeros((1, P.nrow, P.ncol), dtype=float)

        sg = flopy.discretization.StructuredGrid(delr=delr, delc=delc, top=top, botm=botm)
        xycenters = sg.xycenters
        
        cell2d = []
        xcyc = [] # added 
        for n in range(P.nrow*P.ncol):
            l,r,c = sg.get_lrc(n)[0]
            xc = xycenters[0][c]
            yc = xycenters[1][r]
            iv1 = c + r * (P.ncol + 1)  # upper left
            iv2 = iv1 + 1
            iv3 = iv2 + P.ncol + 1
            iv4 = iv3 - 1
            cell2d.append([n, xc, yc, 5, iv1, iv2, iv3, iv4, iv1])
            xcyc.append((xc, yc))
        
        vertices = []
        xa = np.arange(P.x0, P.x1 + P.delx, P.delx)      
        ya = np.arange(P.y1, P.y0 - P.dely/2, -P.dely)

        n = 0
        for j in ya:
            for i in xa:
                vertices.append([n, i, j])
                n+=1
                
        return(cell2d, xcyc, vertices, sg)
    
    if grid == 'tri': 
        boresinner, boresouter, obsinner, obsouter = prepboremesh(P, grid = grid)
        modelextpoly, modelintpoly = prepboundarymesh(P, grid = grid)
        faultpoints = prepfaultmesh(P, grid = grid)

        nodes = []
        for bore in boresinner: 
            for n in bore: nodes.append(n)
        for bore in boresouter: 
            for n in bore: nodes.append(n)
        for bore in obsinner: 
            for n in bore: nodes.append(n)
        for bore in obsouter: 
            for n in bore: nodes.append(n)
        if fault == True:
            for point in faultpoints:
                nodes.append(point)
        
        nodes = np.array(nodes)
        
        polygons = []
        polygons.append((modelextpoly, (P.x0 + 10, P.y0 + 10), P.boundmaxtri)) # Inside boundary frame
        polygons.append((modelintpoly, (P.x0 + P.w + 10, P.y0 + P.w + 10), P.modelmaxtri)) # Bulk of model!       
        
        cell2d, xcyc, vertices, gridobject = tri_meshing(P, polygons, nodes)
        
        return(cell2d, xcyc, vertices, gridobject, nodes)
        
    if grid == 'vor': 
        pumpinner, pumpouter, obsinner, obsouter = prepboremesh(P, grid = grid)
        modelextpoly, modelintpoly = prepboundarymesh(P, grid = grid)
        faultpoints = prepfaultmesh(P, grid = grid)
        
        nodes = []
        #for point in modelextpoly: # Added back 29/4
        #    nodes.append(point) # Added back 29/4
        #for point in modelintpoly: # Added back 29/4
        #    nodes.append(point) # Added back 29/4
        for point in P.xypumpbores:
            nodes.append(point)
        for point in P.xyobsbores:
            nodes.append(point)
        if fault == True:
            for point in faultpoints:
                nodes.append(point)

        nodes = np.array(nodes)
        
        polygons = []
        polygons.append((modelextpoly, (P.x0 + 10, P.y0 + 10), P.boundmaxtri)) # Inside boundary frame
        polygons.append((modelintpoly, (P.x0 + P.w + 10, P.y0 + P.w + 10), P.modelmaxtri)) # Bulk of model!     
        
        for i in range(P.npump): # Append pumping bore zone polygons
            polygons.append((pumpinner[i], P.xypumpbores[i], P.boremaxtri))
            polygons.append((pumpouter[i],0, 0)) # 0, 0 means don't refine inside polygon
            
        for i in range(P.nobs): # Append pumping bore zone polygons
            polygons.append((obsinner[i], P.xyobsbores[i], P.boremaxtri))
            #polygons.append((obsouter[i],0, 0)) # 0, 0 means don't refine inside polygon
        
        cell2d, xcyc, vertices, gridobject = vor_meshing(P, polygons, nodes)
    
        return(cell2d, xcyc, vertices, gridobject, nodes)

def tri_meshing(P, polygons, nodes):

    import flopy
    from flopy.discretization import VertexGrid
    from flopy.utils.triangle import Triangle as Triangle
    
    tri = Triangle(angle=P.angle, model_ws=P.workspace, exe_name=P.triExeName, nodes = nodes,
                   additional_args = ['-j','-D'])

    for poly in polygons:
        tri.add_polygon(poly[0]) 
        if poly[1] != 0: # Flag set to zero if region not required
            tri.add_region(poly[1], 0, maximum_area = poly[2]) # Picks a point in main model

    tri.build(verbose=False) # Builds triangular grid

    cell2d = tri.get_cell2d()     # cell info: id,x,y,nc,c1,c2,c3 (list of lists)
    vertices = tri.get_vertices()
    xcyc = tri.get_xcyc()
    
    return(cell2d, xcyc, vertices, tri)

def vor_meshing(P, polygons, nodes):

    import flopy
    from flopy.discretization import VertexGrid
    from flopy.utils.triangle import Triangle as Triangle
    from flopy.utils.voronoi import VoronoiGrid
    
    tri = Triangle(angle=P.angle, model_ws=P.workspace, exe_name=P.triExeName, nodes = nodes,
                   additional_args = ['-j','-D'])

    for poly in polygons:
        tri.add_polygon(poly[0]) 
        if poly[1] != 0: # Flag set to zero if region not required
            tri.add_region(poly[1], 0, maximum_area = poly[2]) # Picks a point in main model

    tri.build(verbose=False) # Builds triangular grid

    vor = VoronoiGrid(tri)
    vertices = vor.get_disv_gridprops()['vertices']
    cell2d = vor.get_disv_gridprops()['cell2d']

    xcyc = []
    for cell in cell2d:
        xcyc.append((cell[1],cell[2]))
    
    return(cell2d, xcyc, vertices, vor)

In [ ]:
def create_data(P):

    fault_azimuth = np.degrees(np.arcsin((P.fx2-P.fx1)/(P.fy2-P.fy1)))  
    dip_azimuth = fault_azimuth + 90                         
    stratdip = 5  
    H = P.y1 * np.cos(np.radians(dip_azimuth)) * np.tan(np.radians(stratdip))  # strat offset in lower between P1/P2, and P3/P4

    data = {
        'ID': ['P1', 'P1', 'P1', 'P2', 'P2', 'P2', 'P3', 'P3', 'P3', 'P4', 'P4', 'P4'],
        'X': [1, 1, 1, 1, 1, 1,
              P.x1-1, P.x1-1, P.x1-1, P.x1-1, P.x1-1, P.x1-1],
        'Y': [1, 1, 1, P.y1-1, P.y1-1, P.y1-1,
              P.y1-1, P.y1-1, P.y1-1, 1, 1, 1],
        'Z': [-100, -600, -800,
              -100, -600+H, -800+H,
              -100, -110, -130,
              -100, -110-H, -130-H],
        'val': [0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0,],
        'feature_name': ['upper', 'lower', 'lower', 'upper', 'lower', 'lower',
                         'upper', 'lower', 'lower', 'upper', 'lower', 'lower'],
        'gx': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        'gy': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        'gz': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
    }    

    df = pd.DataFrame(data)

    # Make cloud of points along fault plane
    nh = 10 # points  in x/y plane
    x_array, y_array = [], [] # arrays to create points along fault
    
    x_array.append(P.fx1)
    y_array.append(P.fy1)
    for i in range(nh-2):
        x_array.append(P.fx1 + (i+1) * (P.fx2-P.fx1)/(nh-1))
        y_array.append(P.fy1 + (i+1) * (P.fy2-P.fy1)/(nh-1))
    x_array.append(P.fx2)
    y_array.append(P.fy2)
    
    z_array = np.arange(-800, -200, 100) 
    nv = len(z_array) # points in z plane   
 
    strike, dip = fault_azimuth, 90                                                                           

    #from LoopStructural.utils import strikedip2vector # array of arrays
    #nx, ny, nz = strikedip2vector([strike], [dip])[0]
    
    from LoopStructural.utils.helper import strike_dip_vector
    nx, ny, nz = strike_dip_vector([strike], [dip])[0]

    fault_plane_3d = []
    for v in range(nv):# vertical points 
        for h in range(nh): # horizontal points
            x, y, z = x_array[h], y_array[h], z_array[v]
            fault_plane_3d.append((x,y,z))
            df_new_row = pd.DataFrame.from_records({'X':[x], 'Y':[y], 'Z':[z], 'val':[0.], 'feature_name':['Fault'], 'nx': [nx], 'ny': [ny], 'nz': [nz]})
            df = pd.concat([df, df_new_row], ignore_index = True)
        
    return(df)

In [ ]:
def create_faultfunction():    
    ## ADD FAULT (this chunk given to me directly by Lachlan Grose to make an ellipsoid fault)
    from LoopStructural.modelling.features.fault._fault_function import CubicFunction, FaultDisplacement, Composite
    hw = CubicFunction()
    hw.add_cstr(0, 1)
    hw.add_grad(0, 0)
    hw.add_cstr(1, 0)
    hw.add_grad(1, 0)
    hw.add_max(1)
    fw = CubicFunction()
    fw.add_cstr(0, -1)
    fw.add_grad(0, 0)
    fw.add_cstr(-1, 0)
    fw.add_grad(-1, 0)
    fw.add_min(-1)
    gyf = CubicFunction()
    gyf.add_cstr(-1, 0)
    gyf.add_cstr(1, 0)
    gyf.add_cstr(-0.2, 1)
    gyf.add_cstr(0.2, 1)
    gyf.add_grad(0, 0)
    gyf.add_min(-1)
    gyf.add_max(1)
    gzf = CubicFunction()
    gzf.add_cstr(-1, 0)
    gzf.add_cstr(1, 0)
    gzf.add_cstr(-0.2, 1)
    gzf.add_cstr(0.2, 1)
    gzf.add_grad(0, 0)
    gzf.add_min(-1)
    gzf.add_max(1)
    gxf = Composite(hw, fw)
    fault_displacement = None
    fault_displacement = FaultDisplacement(gx=gxf, gy=gyf, gz=gzf)
    faultfunction = fault_displacement
    return(faultfunction)

In [ ]:
def create_geomodel(P, df):
    origin  = (P.x0, P.y0, P.z0)
    maximum = (P.x1, P.y1, P.z1)
    faultfunction = create_faultfunction()

    from LoopStructural import GeologicalModel
    geomodel = GeologicalModel(origin, maximum)
    geomodel.set_model_data(df)

    upper = geomodel.create_and_add_foliation("upper")   

    Fault = geomodel.create_and_add_fault('Fault', 
                                          displacement = P.fault_max_disp,
                                          fault_slip_vector = P.fault_slip_vector,
                                          fault_center      = P.fault_center,
                                          minor_axis        = P.minor_axis, # fault_influence
                                          major_axis        = P.major_axis, # fault_extent
                                          intermediate_axis = P.intermediate_axis, # fault_vertical_radius
                                          #faultfunction     = faultfunction, #'BaseFault', 
                                          nelements=4000, 
                                          steps=4, 
                                          interpolatortype="FDI", 
                                          buffer=0.3, 
                                          solver='cg',
                                          )  

    uc = geomodel.add_unconformity(upper, 0) # Clips above
    lower = geomodel.create_and_add_foliation("lower")

    # ADD STRAT COLUMN
    stratigraphic_column = {}
    stratigraphic_column['upper'] = {}
    stratigraphic_column['upper']['overburden'] = {'min':0,'max':np.inf,'id':0}
    stratigraphic_column['lower'] = {}
    stratigraphic_column['lower']['upper_aquifer'] = {'min':1,'max':np.inf,'id':1}
    stratigraphic_column['lower']['aquitard'] = {'min':0,'max':1,'id':2}
    stratigraphic_column['lower']['lower_aquifer'] = {'min':-np.inf,'max':0,'id':3}

    geomodel.set_stratigraphic_column(stratigraphic_column)
    
    return(geomodel)

In [ ]:
def prepare_geomodel_loopshowcase(P, include_fault):
    
    bore_data = pd.read_excel("loop_showcase_data.xls",sheet_name = "bore_data")
    strat = pd.read_excel("loop_showcase_data.xls",sheet_name = "strat")

    df = bore_data.copy()
    units = list(df.columns.values[4:])   # Make a list of formations  
    df.easting = pd.to_numeric(df.easting)    # Make sure Eastings and Northings are numeric values
    df.northing = pd.to_numeric(df.northing)
    df.ground = pd.to_numeric(df.ground)


    # ---------- Prepare borehole data ----------------
    data_list = df.values.tolist()  # Turn data into a list of lists
    formatted_data = []
    for i in range(len(data_list)): #iterate for each row

        #val = strat.val[0] # Add data for groundlevel
        #gx, gy, gz = 0,0,1 # flat
        #formatted_data.append([boreid, easting, northing, groundlevel, val, 'ground', 'ground', gx, gy, gz]) 
        
        boreid = data_list[i][0]
        easting, northing = data_list[i][2], data_list[i][3]
        groundlevel = data_list[i][4]  
        count = 0  # Add data row for each lithology
        for j in range(5,df.shape[1]-1): #iterate through each formation 
            if isinstance(data_list[i][j], numbers.Number) == True:  # Add lithology  
                bottom    = groundlevel + float(data_list[i][j])  # Ground surface - formation bottom (mbgl)
                val       = strat.val[count]                   # designated isovalue
                unit      = strat.unit[count]                  # unit 
                feature   = strat.sequence[count]              # sequence
                gx, gy, gz = 0,0,1                             # normal vector to surface (flat)
                formatted_data.append([boreid, 'raw', easting, northing, bottom, val, unit, feature, gx, gy, gz])    
                current_bottom = np.copy(bottom)            
            count+=1

    # ---------- Add control points ----------------   
   
    for cp in P.control_points:
            formatted_data.append(cp)
    
    data = pd.DataFrame(formatted_data)
    data.columns =['ID','data_type','X','Y','Z','val','lithcode','feature_name', 'gx', 'gy', 'gz']
    
    # ---------- Prepare fault details ----------------   
    if include_fault:
        # Make cloud of points along fault plane
        nh = 10 # points  in x/y plane
        x_array, y_array = [], [] # arrays to create points along fault
        x_array.append(P.fx1)
        y_array.append(P.fy1)
        for i in range(nh-2):
            x_array.append(P.fx1 + (i+1) * (P.fx2-P.fx1)/(nh-1))
            y_array.append(P.fy1 + (i+1) * (P.fy2-P.fy1)/(nh-1))
        x_array.append(P.fx2)
        y_array.append(P.fy2)

        z_array = np.arange(-200, -100, 20) 
        nv = len(z_array) # points in z plane   

        fault_azimuth = np.degrees(np.arcsin((P.fx2-P.fx1)/(P.fy2-P.fy1)))  
        strike, dip = fault_azimuth-180, 90                                                                           

        from LoopStructural.utils import strikedip2vector # array of arrays
        nx, ny, nz = strikedip2vector([strike], [dip])[0]
        #from LoopStructural.utils.helper import strike_dip_vector
        #nx, ny, nz = strike_dip_vector([strike], [dip])[0]

        fault_plane_3d = []
        for v in range(nv):# vertical points 
            for h in range(nh): # horizontal points
                x, y, z = x_array[h], y_array[h], z_array[v]
                fault_plane_3d.append((x,y,z))
                df_new_row = pd.DataFrame.from_records({'ID':['fault'],'data_type':['fault_surface'],
                                                        'X':[x], 'Y':[y], 'Z':[z], 'val':[0.], 
                                                        'feature_name':['Fault'], 'nx': [nx], 'ny': [ny], 'nz': [nz]})
                data = pd.concat([data, df_new_row], ignore_index = True)
            
    return(data, strat)


def create_geomodel_loopshowcase(P, include_fault):
    origin  = (P.x0, P.y0, P.z0)
    maximum = (P.x1, P.y1, P.z1)
    
    from LoopStructural import GeologicalModel
    geomodel = GeologicalModel(origin, maximum)
    geomodel.set_model_data(P.data)  
    
    Upper      = geomodel.create_and_add_foliation("upper")
    UpperUC    = geomodel.add_unconformity(Upper, -50) 
    Lower      = geomodel.create_and_add_foliation("lower") 
    print(geomodel.feature_name_index)

    if include_fault:
        faultfunction = create_faultfunction()
        Fault = geomodel.create_and_add_fault('Fault', 
                                              displacement = P.fault_max_disp,
                                              fault_slip_vector = P.fault_slip_vector,
                                              fault_center      = P.fault_center,
                                              minor_axis        = P.minor_axis, # fault_influence
                                              major_axis        = P.major_axis, # fault_extent
                                              intermediate_axis = P.intermediate_axis, # fault_vertical_radius
                                              fault_dip_anisotropy=0.0,
                                              fault_trace_anisotropy=0.0,
                                              #faultfunction     = faultfunction, #'BaseFault', 
                                              nelements=4000, 
                                              steps=4, 
                                              interpolatortype="FDI", 
                                              buffer=0.3, 
                                              solver='cg',
                                              )    
    
    # Add Strat Column
    stratigraphic_column = {}
    stratigraphic_column["upper"] = {}
    stratigraphic_column["upper"]["a"] = {"min": -50, "max": np.inf, "id": 0}
    stratigraphic_column["lower"] = {}
    stratigraphic_column["lower"]["b"] = {"min": -100, "max": np.inf, "id": 1}
    stratigraphic_column["lower"]["c"] = {"min": -200, "max": -100, "id": 2}
    stratigraphic_column["lower"]["d"] = {"min": -np.inf, "max": -200, "id": 3}


    geomodel.set_stratigraphic_column(stratigraphic_column)
    
    #from LoopStructural.visualisation import LavaVuModelViewer
    #view =  LavaVuModelViewer(geomodel)
    #view.add_model_surfaces(cmpa = 'Spectral', faults=False)
    #view.set_zscale(zscale = 10)
    #view.interactive()  
    
    return(geomodel)

In [ ]:
def plot_bores(P):
    plt.figure(figsize=(3,3))
    
    plt.xlabel('Easting')
    plt.ylabel('Northing')
    i = 0
    for (xi, yi) in zip(P.data.X, P.data.Y):
        if P.data.data_type[i] == 'raw':
            plt.scatter(P.data.X[i], P.data.Y[i], color = 'blue')#, size = 2)
            plt.text(xi, yi, P.data.ID[i], size = 11, va='bottom', ha='center')
        if P.data.data_type[i] == 'control':
            plt.scatter(P.data.X[i], P.data.Y[i], color = 'red')#, size = 2)
            plt.text(xi, yi, P.data.ID[i], size = 11, va='bottom', ha='center')
        if P.data.data_type[i] == 'fault_surface':
            plt.scatter(P.data.X[i], P.data.Y[i], color = 'purple')#, size = 1)
        i += 1
    plt.xlim(P.x0,P.x1)
    plt.ylim(P.y0,P.y1)
        

def plot_geo_2D(geomodel, X, upper_levels, lower_levels):
    import matplotlib.pyplot as plt

    fig, ax = plt.subplots(1,1,figsize=(5,2))
    r = 200
    # X section
    y = np.linspace(P.y0,P.y1,r)
    z = np.linspace(P.z0,P.z1,r)
    yy, zz = np.meshgrid(y,z)
    xx = np.zeros_like(yy)
    xx[:] = X
    ax.set_title('X = ' + str(X))

    vals = geomodel.evaluate_feature_value('upper',np.array([xx.flatten(),yy.flatten(),zz.flatten()]).T)
    ax.contourf(vals.reshape((r,r)), levels = upper_levels, extent=(P.y0, P.y1, P.z0, P.z1), cmap = 'Spectral')
    vals = geomodel.evaluate_feature_value('lower',np.array([xx.flatten(),yy.flatten(),zz.flatten()]).T)
    ax.contourf(vals.reshape((r,r)), levels = lower_levels, extent=(P.y0, P.y1, P.z0, P.z1))

    plt.show()
    


In [ ]:
print('Modelling routines run!')